### library 설치 (colab 기준)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd "/content/drive/MyDrive/KT/BM25"

/content/drive/MyDrive/KT/BM25


In [5]:
!pip install konlpy
!pip install kiwipiepy
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602629 sha256=e2bf5d13286b5929b60f9f529e2e4dac5d6e5353b495e02c8a78cfa7e7c10644
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


## BM25 모델링
bm25.py 내 구현해놓은 함수 기능 간단하게 보여주기

### 데이터 불러오기

In [ ]:
import os
from rank_bm25 import BM25Okapi
from bs4 import BeautifulSoup as bs
from kiwipiepy import Kiwi

In [ ]:
## article에서 text만 추출하여 string list로 저장

pth = "./articles/"
articles_id = os.listdir(pth)
articles_id.sort()

articles = []

for id in articles_id:
    f = open(pth + id, 'r')
    html = f.read()
    f.close()

    article = bs(html, 'html.parser')
    article = " ".join(article.text.replace("\n", "").split())
    articles.append(article)

articles[0]

'사회서비스원 긴급돌봄 사업 1. 대상 구분 지원대상 코로나19 긴급돌봄 · 아동, 노인, 장애인 등 돌봄이 필요하나, 확진으로 인한 보호자의 입원 등으로 가정 내 돌봐줄 사람이 부재한 경우 · 본인의 자가격리 등 코로나19 확산으로 이용하고 있던 기존 돌봄서비스가 중단됐으나 돌봄을 제공할 보호자 및 대체 서비스가 부재한 경우 · 종사자 확진 또는 백신 접종 후 이상 반응 등에 따른 종사자 휴가 등으로 사회복지시설 내 돌봄 인력 공백이 생긴 경우 · 시·도 또는 긴급돌봄지원단 내 대상자 선정위원회에서 코로나19로 인해 긴급하게 돌봄이 필요하다고 인정하는 경우 코로나19 외 긴급돌봄 · 갑작스러운 질병(수술)·사고, 일시적 신체 저하로 긴급하게 재가·이동지원 등 돌봄서비스가 필요한 자 · 일시적으로 돌봄이 필요하나 가족·부양의무자가 돌봄을 제공하기 어려운 경우 · 장애등급 및 장기요양등급 판정 등 기존 제도권 서비스 대상자 선정 대기 중단기적으로 서비스가 필요한 경우 · 기타 위기 사유로 인해 긴급한 돌봄이 필요하다고 시·도에서 인정하는 경우 2. 내용 시도지사, 시군구청장 또는 긴급돌봄지원단 대상자 선정위원회에서 선정한 긴급돌봄 대상자에게 긴급돌봄서비스를 제공할 수 있는 인력을 매칭 지원 구분 지원내용 코로나19 긴급돌봄 의료기관, 생활치료센터 입소기간, 자가격리 기간 등을 고려하여 최대 7일을 원칙으로 하되, 긴급한 필요가 있는 경우 3일 연장 가능 코로나19 외 긴급돌봄 일시적인 위기상황 1회당 총 100시간(30일 내) 지원, 불가피한 경우에 한정하여 연장 가능 3. 방법 시도 또는 시군구, 사회서비스원에 상담 및 신청 4. 문의 사회서비스원 종합재가센터(대표번호 ☎1522-0365)'

### 추가 전처리

In [ ]:
### 특수문자 제거
import re

def clean_str(text):
    text = re.sub('☎([0-9]+-?[0-9]+)', " ", text)
    text = re.sub('[^\w\s\n]', " ", text)
    return text

articles = [clean_str(article) for article in articles]
articles[0]

'사회서비스원 긴급돌봄 사업 1  대상 구분 지원대상 코로나19 긴급돌봄   아동  노인  장애인 등 돌봄이 필요하나  확진으로 인한 보호자의 입원 등으로 가정 내 돌봐줄 사람이 부재한 경우   본인의 자가격리 등 코로나19 확산으로 이용하고 있던 기존 돌봄서비스가 중단됐으나 돌봄을 제공할 보호자 및 대체 서비스가 부재한 경우   종사자 확진 또는 백신 접종 후 이상 반응 등에 따른 종사자 휴가 등으로 사회복지시설 내 돌봄 인력 공백이 생긴 경우   시 도 또는 긴급돌봄지원단 내 대상자 선정위원회에서 코로나19로 인해 긴급하게 돌봄이 필요하다고 인정하는 경우 코로나19 외 긴급돌봄   갑작스러운 질병 수술  사고  일시적 신체 저하로 긴급하게 재가 이동지원 등 돌봄서비스가 필요한 자   일시적으로 돌봄이 필요하나 가족 부양의무자가 돌봄을 제공하기 어려운 경우   장애등급 및 장기요양등급 판정 등 기존 제도권 서비스 대상자 선정 대기 중단기적으로 서비스가 필요한 경우   기타 위기 사유로 인해 긴급한 돌봄이 필요하다고 시 도에서 인정하는 경우 2  내용 시도지사  시군구청장 또는 긴급돌봄지원단 대상자 선정위원회에서 선정한 긴급돌봄 대상자에게 긴급돌봄서비스를 제공할 수 있는 인력을 매칭 지원 구분 지원내용 코로나19 긴급돌봄 의료기관  생활치료센터 입소기간  자가격리 기간 등을 고려하여 최대 7일을 원칙으로 하되  긴급한 필요가 있는 경우 3일 연장 가능 코로나19 외 긴급돌봄 일시적인 위기상황 1회당 총 100시간 30일 내  지원  불가피한 경우에 한정하여 연장 가능 3  방법 시도 또는 시군구  사회서비스원에 상담 및 신청 4  문의 사회서비스원 종합재가센터 대표번호   '

In [ ]:
### tokenize -> kiwi 적용 결과
tokenizer = Kiwi()
tokenizer.prepare()
tokenized_articles = [tokenizer.analyze(article) for article in articles]
tokenized_articles[0][0][0][:5]

[Token(form='사회', tag='NNG', start=0, len=2),
 Token(form='서비스', tag='NNG', start=2, len=3),
 Token(form='원', tag='NNG', start=5, len=1),
 Token(form='긴급', tag='NNG', start=7, len=2),
 Token(form='돌봄', tag='NNG', start=9, len=2)]

In [ ]:
out_pos=['JKO', 'JKG', 'JKS', 'JKB', 'VCP', 'EC', 'EF', 'JX'] #품사 선택
# out_pos = ['test']
bm25_list = []
min_word = 2

## 특정 품사 선택, 일정 길이 미만 단어 제외, (숫자로 이루어진 단어 제외)

for i in range(len(tokenized_articles)):
    temp_word = []
    article = tokenized_articles[i][0][0]
    for j in range(len(article)):
        if article[j][1] not in out_pos and len(str(article[j][0])) >= min_word:
            temp_word.append(article[j][0])
    bm25_list.append(temp_word)

bm25_list[0][:5]

['사회', '서비스', '긴급', '돌봄', '사업']

### bm25 적용

In [ ]:
bm25 = BM25Okapi(bm25_list)

In [ ]:
query = "폭행 피해 여성"
tokenized_query = tokenizer.analyze(query)
tokenized_query = [tokenized_query[0][0][i][0] for i in range(len(tokenized_query[0][0]))]
score = bm25.get_scores(tokenized_query)
score[:10]

array([0.        , 0.        , 0.        , 0.        , 2.15933334,
       0.        , 0.        , 2.31342948, 0.        , 2.59941067])

In [ ]:
bm25.get_top_n(tokenized_query, articles, 5)

['폭력 피해여성 주거 지원  대상 가정폭력 성폭력 피해 여성이주여성 포함 및 동반 가족  내용 폭력 피해여성과 그 가족들이 공동으로 생활할 수 있는 주거공간 제공  공동생활가정형매입임대  년에서 연장 시 최대 년까지 지원  방법 거주지 읍면동 주민센터행정복지센터 또는 운영기관에 신청  문의  여성긴급전화  각종 공공임대주택  LH마이홈',
 '가정폭력 피해자 지원  대상 가정폭력 피해자 및 동반가족스토킹 피해자 이용 가능  내용 구분 지원내용 가정폭력 상담소 피해신고상담 임시보호 의료기관 연계서비스 등 지원 가정폭력 피해자 보호시설 피해자와 동반자녀 보호 및 숙식제공 상담 직업훈련 피해 회복프로그램 등 지원 주거 지원 임대주택 공급으로 피해여성과 동반가족에게 주거공간 제공  방법 가정폭력 피해 발생  여성긴급전화  긴급상담 서비스 연계  가정폭력상담소 상담 지원  의료기관 의료 지원  무료법률구조기관 법률 지원  경찰검찰 수사 지원  가정폭력 피해자 보호시설 보호자립 지원  문의  여성긴급전화  여성폭력 사이버상담wwwwomenkr  카카오톡 IDwomen  전국 가정폭력상담소  여성폭력 사이버상담wwwwomenkr  여성폭력피해자 지원기관 안내  가정폭력상담소',
 '성매매 피해자 지원  대상 성매매 피해를 입은 일반성인청소년외국인 등  내용 구분 지원내용 성매매피해상담소 긴급구조 현장방문상담 의료법률문제 지원 쉼터 연계 등 성매매피해자 쉼터 성인 청소년 외국인 지원시설 숙식제공 전문상담 의료법률문제 지원 직업훈련취업 연계 진학교육  외국인여성 지원시설의 경우 통역서비스전문상담자국 대사관 등 연계귀국지원 등 추가 제공 자립지원공동생활시설 일정기간 주거지원 등 자활지원센터 인턴십프로그램 연계 등 일자리제공 직업훈련진학교육 취업지원 등 대안교육위탁기관 청소년 성매매 피해자 대상 의무교육 및 자활교육 사회적응훈련 등  방법 피해 발생 상담  긴급구조 ⇦ 여성긴급 전화  상담연계 연계 \uf0f3 상담 연계 \uf0f3 보호 연계 \uf0f3 자활  성매매 재유입

## module 사용법

In [7]:
cd "/content/drive/MyDrive/KT/code"

/content/drive/MyDrive/KT/code


In [ ]:
from bm25 import BM25

In [ ]:
# data 다시 전처리 진행
bm25 = BM25('/content/drive/MyDrive/KT/data/articles/')

In [ ]:
# 기존에 전처리해둔 data 사용
bm25 = BM25(articles_path = '/content/drive/MyDrive/KT/data/articles/',
             update_data = False)

### query 돌려보기

In [ ]:
import json
import numpy as np

with open("/content/drive/MyDrive/KT/data/augmented/final.json") as f:
    data = json.load(f)

In [ ]:
### example
data['기타지원_05.html']

{'index': 4,
 'title': '청소년한부모 등 자립지원패키지',
 'questions': ['나는 혼자 아이를 양육하고 책임을 다루는데 어려움을 겪고 있어',
  '나는 제한된 자원을 가진 젊은 미혼 엄마이며 임신 및 출산 후 보호에 도움이 필요해',
  '나는 최근에 싱글 부모가 되어 안정된 집을 제공하기 위해 주거 지원이 필요해',
  '나는 젊은 싱글 어머니이며 아이를 경제적으로 지원하기 위해 취업 기회가 필요해',
  '젊은 싱글 부모로서 압도되는 느낌이 들어 상담이나 치료가 정말 필요해']}

In [ ]:
score = bm25.get_score("나는 혼자 아이를 양육하고 책임을 다루는데 어려움을 겪고 있어")
index = np.array(range(462))

In [ ]:
index[score.argsort()[::-1]]

array([  9,  56, 263, 259, 216,  38, 353, 261, 378,  57, 379, 112, 385,
       265, 153, 448, 301, 317,  61, 133, 358, 373, 217, 267, 294, 321,
       372,  88, 457, 295, 232, 332,  32,  54, 215, 357, 426, 291,  52,
       393,  10,  49, 322,  84, 113, 260, 432, 459, 389,   0,  87, 154,
       124, 290,  46, 394, 436, 235, 320, 360, 247, 416, 318, 202, 104,
       437, 195, 250, 203,  37, 231, 140, 115, 178, 230, 148, 401, 171,
       345, 388, 262, 246, 241,  74, 101,  51,  43, 313, 249, 356, 441,
       404, 252, 392, 107, 289, 421,  39, 450, 336, 198, 103, 189, 256,
       224, 310, 129,  19, 197,  98, 123, 121,  99,  80, 150, 315, 355,
       238,   4, 323, 335, 391, 362, 283, 149,  58,  85,  47,  83, 152,
       193,   5, 126, 381, 240,  42, 331, 254, 172, 209, 438, 132,   8,
        50, 191, 430,  70,  33,  82, 137,  28, 292, 156, 324, 139, 409,
       364, 117, 248, 444,  71, 188, 173, 407, 303, 135, 374, 377, 114,
       305, 369, 244, 316, 266, 422, 206, 337, 251,  55, 205, 38

In [9]:
ex.
! python bm25.py

[ 0.          0.          0.          0.          4.85000165  3.06279348
  3.3379145   0.          4.15087289  8.63420538  2.31095115  0.
  0.          0.          0.          0.          0.          0.
  0.          3.4843445   3.0140954   0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          5.38207946  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          2.87465673  0.          0.          0.
  0.          4.88893379  0.          6.25654096  0.          0.
  0.          2.12422221  0.          3.3379145   0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0. 